# Quantization and Dequantization in 1-D tensors

Implementing simple quantization and dequantization on arrays using min-max values of tensor for quantization range

In [1]:
import numpy as np

In [2]:
# Randomly distributed parameters
parameters = np.random.uniform(low=-50, high=150, size=20)

# Round each number of second decimal place
parameters = np.round(parameters, 2)
print(parameters)

[ 79.63 100.75  47.9   85.26 -19.68  91.28  82.59  -7.12  90.24  60.92
  67.78  27.35 115.33 143.   104.42  84.64 146.59  51.34 125.74 126.03]


Defining quantization methods and implementing quantization

In [3]:
def clamp(param_q, lower_bound, upper_bound):
    """
    Function to implement clamp
    """
    param_q[param_q < lower_bound] = lower_bound
    param_q[param_q > upper_bound] = upper_bound
    return param_q

In [5]:
def asymetric_quantization(parameters, bits):
    """
    Assymetric quantization
    """
    alpha = np.max(parameters)
    beta = np.min(parameters)
    scale = (alpha - beta) / (2**bits-1)
    zero = -1*np.round(beta/scale)
    lower_bound = 0
    upper_bound = 2**bits-1
    quantized_params = clamp(np.round(parameters/scale+zero), lower_bound, upper_bound).astype(np.int32)
    return quantized_params, scale, zero

def asymetric_dequantization(param_q, scale, zero):
    return (param_q - zero) * scale

In [8]:
def symmetric_quantization(parameters, bits):
    alpha = np.max(np.abs(parameters))
    scale = (alpha) / (2**(bits-1)-1)
    lower_bound = -2**bits-1
    upper_bound = 2**(bits-1)-1
    quantized_params = clamp(np.round(parameters/scale), lower_bound, upper_bound).astype(np.int32)
    return quantized_params, scale

def symmetric_dequantization(param_q, scale):
    return param_q * scale

In [9]:
def quantization_error(parameters, param_q):
    return np.mean((parameters - param_q)**2)

In [10]:
(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymetric_quantization(parameters, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(parameters, 8)

print(f'Original:')
print(np.round(parameters, 2))
print('')
print(f'Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}')
print(asymmetric_q)
print('')
print(f'Symmetric scale: {symmetric_scale}')
print(symmetric_q)

Original:
[ 79.63 100.75  47.9   85.26 -19.68  91.28  82.59  -7.12  90.24  60.92
  67.78  27.35 115.33 143.   104.42  84.64 146.59  51.34 125.74 126.03]

Asymmetric scale: 0.6520392156862745, zero: 30.0
[152 185 103 161   0 170 157  19 168 123 134  72 207 249 190 160 255 109
 223 223]

Symmetric scale: 1.154251968503937
[ 69  87  41  74 -17  79  72  -6  78  53  59  24 100 124  90  73 127  44
 109 109]


In [11]:
# Dequantize the parameters back to 32 bits
params_deq_asymmetric = asymetric_dequantization(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantization(symmetric_q, symmetric_scale)

print(f'Original:')
print(np.round(parameters, 2))
print('')
print(f'Dequantize Asymmetric:')
print(np.round(params_deq_asymmetric,2))
print('')
print(f'Dequantize Symmetric:')
print(np.round(params_deq_symmetric, 2))

Original:
[ 79.63 100.75  47.9   85.26 -19.68  91.28  82.59  -7.12  90.24  60.92
  67.78  27.35 115.33 143.   104.42  84.64 146.59  51.34 125.74 126.03]

Dequantize Asymmetric:
[ 79.55 101.07  47.6   85.42 -19.56  91.29  82.81  -7.17  89.98  60.64
  67.81  27.39 115.41 142.8  104.33  84.77 146.71  51.51 125.84 125.84]

Dequantize Symmetric:
[ 79.64 100.42  47.32  85.41 -19.62  91.19  83.11  -6.93  90.03  61.18
  68.1   27.7  115.43 143.13 103.88  84.26 146.59  50.79 125.81 125.81]


In [12]:
# Quantization error
print(f'{"Asymmetric error: ":>20}{np.round(quantization_error(parameters, params_deq_asymmetric), 2)}')
print(f'{"Symmetric error: ":>20}{np.round(quantization_error(parameters, params_deq_symmetric), 2)}')

  Asymmetric error: 0.03
   Symmetric error: 0.1
